# Final Analysis of Vaccination Rates
Vaccination rates have begun to fall in areas of Califonia. This has led to outbreaks of previously erradicated diseases. This isn't just a problem in California so it would be helpful to identify demographic indicators that could help us target where outbreaks are more likely to occur, allowing targeted educaitonal campaigns to combat misinformation about vaccinations.
## Exploration Questions
What does the vaccination rate look like accross California?

What relationship (if any) do the folowing variables have on the vaccination rates?accross countries.
- Income
- Unemployment
- County Education Level
- Insured Rate
- County Population

## Data being used
- Kindergarten Immunization records from Kaggle [link](https://www.kaggle.com/broach/california-kindergarten-immunization-rates)
- CA census estimates currated by the state [link](http://www.dof.ca.gov/Reports/Demographic_Reports/American_Community_Survey/#ACS2017x5)

# Imports

## Modules and Environment

In [1]:
# Calculation
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Modules
import subprocess
from functools import reduce

get_ipython().run_line_magic('config', 'IPCompleter.greedy = True')

## Data from exploration and cleanup

In [2]:
from helper_funcs import merged_df

ModuleNotFoundError: No module named 'helper_funcs'

In [ ]:
merged_df.head()

# Visualizations and Analysis

In [ ]:
# Some Descriptive data and Probabability density function, looking for outliers
num_of_bins = 20
display(pd.DataFrame(merged_df['Percent Vaccinated'].describe()))
ax = sns.distplot(merged_df['Percent Vaccinated'], bins=num_of_bins)
fig = ax.get_figure()
fig.savefig('output/vaccination_distplot.png', dpi=300)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.barplot(x='County', y='Population', data=merged_df, ax=ax)
ax.tick_params(rotation=90)
ax.set_title('County Populations')
fig.tight_layout()
fig.savefig('output/population_bar.png', dpi=300)

## Education level potentially has a correlation

In [ ]:
education_cols = ['No High School Diploma', 'High School or Higher',
                 'Bachelors or Higher']
sns_plot = sns.pairplot(merged_df, y_vars='Percent Vaccinated', x_vars=education_cols,
                 kind='reg', height=5)
sns_plot.savefig('output/education_bubble.png', dpi=300)

## Linear regression on no High School Diploma
There is a significantly small p-value indicating a correlation. The r-squared value of 0.38 implies reasonable predictive value.

In [ ]:
# Look at linear regression for the High Schol or Higher group
X = merged_df['No High School Diploma']
Y = merged_df['Percent Vaccinated']

X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
predication = model.predict(X)

# print_model = model.summary()
# print(print_model)

const, coef = model.params
p_val = model.pvalues['No High School Diploma']
print(f'The coeficient of our linear Regression is: {round(coef,2)} with a constant of: {round(const,2)}\
 and an R-squared value of: {round(model.rsquared, 2)} and a p-value of: {p_val}.')

# Plot the data with the regression line
fig, ax = plt.subplots()

# Bubble Plot
merged_df.plot.scatter(x='No High School Diploma', y='Percent Vaccinated', ax=ax)

# Regression line
x = np.linspace(0, 40)
y = coef * x + const

plt.plot(x, y, '-r')

fig.tight_layout()
fig.savefig('output/lin_reg.png', dpi=300)

## Suprisingly Not Relevant
Here we see that income, unemployment, population, and percent insured seem to have no relevant correlation.

*Note that I ran a regression on Percent Insured found that the p-value was high.

In [ ]:
cols = ['Population', 'Percent Insured', 'Median Income', 'Unemployment Percentage']
sns_plot = sns.pairplot(merged_df, y_vars='Percent Vaccinated', x_vars=cols, kind='reg', height=5)
sns_plot.savefig('output/not_relevant_scatter.png', dpi=300)

# Findings

## What does the vaccination rate look like around California?
- There is a relatively large range of vaccination rates accross California Counties. The difference from the lowest to highest county is about 20%
- The mean and median are in the 95-96% range. This is good for herd immunity, but the Std Dev of 3 means that there are a significant number of counties that are likely below the threshholds for herd immunity needed for contagious diseases like the measles.

## Education level
- There appears to be correlation between a counties percentage of residents with 'No High School Diploma' and the vaccination rate.
    - The high r-squared value implies that this isn't predictive. 
    - It could also be an artifact.
    - We're looking at a .38% increase in percent vacinated for every 1% increase in the percent of high school dropouts.
- Interestingly this correlation seems to dissapear at any other educational milestone.


## Other Demographic indicators
- None of the other indicators I looked at appear to correlate with the vaccination rate.

# Next Steps

## Disaggregate data by city
Noting the large variance in county size, it would be interesting to see if looking at the same data at the city level would lend any new insghts.

## Other demographic indicators
While dissagregating the data might yields some interesting insights, we might just be missing the relevant indicators. Some interesting data to include for further analysis might be:
- Number of members in households
- Ethnicity
- Religion